### This notebook will be used to clean the data and make new columns, and output a final df

In [1]:
import pandas as pd
import numpy as np
app_fname = 'student_applications.csv'
emp_fname = 'generated_handshake_employers2019-03-19T1157.csv'

In [2]:
df_app = pd.read_csv(app_fname)
df_emp = pd.read_csv(emp_fname)

##### Making new columns : application - app start date, due date- application

In [3]:
df_app.columns

Index(['Unnamed: 0', 'Applicant (student) Username',
       'Applications Application Type', 'Applications Created At Date',
       'Employer Name', 'Employer Industry Name', 'Job Locations City',
       'Job Locations Country', 'Job Type Name', 'Postings Apply Start Date',
       'Postings Expiration Date Date', 'Student Educations Cumulative Gpa',
       'Student Educations College Name', 'Student Majors Name',
       'Student School Years Name', 'Applications Count', 'days after posted',
       'days before due'],
      dtype='object')

In [4]:
df_emp.columns

Index(['Employers Name', 'Industry Name', 'Job Postings Apply Start Date',
       'Job Postings Created At Date', 'Job Postings Expiration Date Date',
       'Location City', 'Location Country'],
      dtype='object')

In [5]:
def diff_dates(df, recent_date,old_date):
    df[recent_date] = pd.to_datetime(df[recent_date])
    df[old_date] = pd.to_datetime(df[old_date])
    new_col = df[recent_date] - df[old_date]
    new_col= new_col.dt.days
    
    return new_col

df_app['days after posted']= diff_dates(df_app,'Applications Created At Date','Postings Apply Start Date')
df_app['days before due'] =diff_dates(df_app,'Postings Expiration Date Date','Applications Created At Date')

In [6]:
df_app.shape

(5000, 18)

In [7]:
def make_dummy_var(feat_list,df):
    """
    This makes new binary variable for the given
    categorical variable feature in feat_list
    """
    for feat in feat_list:
        df = pd.concat([df.drop(feat, axis=1), pd.get_dummies(df[feat],prefix = feat)], axis=1)
        
    return df

In [8]:
student_feat_num_list = ['Employer Industry Name','Job Type Name',
                         'Student School Years Name','Student Majors Name']

In [9]:
df_app_numeric = make_dummy_var(student_feat_num_list,df_app)

In [10]:
col_corr = []
for col in df_app_numeric.columns:
    for feat in student_feat_num_list:
        if feat in col:
            col_corr.append(col)
            
col_corr = col_corr+ ['days after posted','days before due','Student Educations Cumulative Gpa']

In [11]:
import seaborn as sns
%matplotlib inline
corr = df_app_numeric[col_corr].corr().abs()

In [12]:
to_drop = [column for column in corr.columns if any(corr[column] >0.5)]
corr1= corr.drop(columns = to_drop)
# plot the heatmap

sns.heatmap(corr1, 
        xticklabels=corr1.columns,
        yticklabels=corr1.index)

ValueError: zero-size array to reduction operation minimum which has no identity

In [78]:
corr.max

<bound method DataFrame.max of                                                     Employer Industry Name_Accounting  \
Employer Industry Name_Accounting                                            1.000000   
Employer Industry Name_Advertising, PR & Marketing                           0.012000   
Employer Industry Name_Aerospace                                             0.008646   
Employer Industry Name_Automotive                                            0.007144   
Employer Industry Name_Biotech & Life Sciences                               0.010551   
Employer Industry Name_CPG - Consumer Packaged ...                           0.004058   
Employer Industry Name_Civil Engineering                                     0.007336   
Employer Industry Name_Commercial Banking & Credit                           0.007796   
Employer Industry Name_Computer Networking                                   0.002341   
Employer Industry Name_Construction                                          0.

### No correlations found !!!

In [13]:
df_app_numeric.shape

(5000, 164)

In [18]:
df_app.columns

Index(['Unnamed: 0', 'Applicant (student) Username',
       'Applications Application Type', 'Applications Created At Date',
       'Employer Name', 'Employer Industry Name', 'Job Locations City',
       'Job Locations Country', 'Job Type Name', 'Postings Apply Start Date',
       'Postings Expiration Date Date', 'Student Educations Cumulative Gpa',
       'Student Educations College Name', 'Student Majors Name',
       'Student School Years Name', 'Applications Count', 'days after posted',
       'days before due'],
      dtype='object')

In [22]:
student_grp = df_app_numeric.groupby(by='Applicant (student) Username').mean().reset_index()

In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(student_grp.mean())

Unnamed: 0                                                    2681.965746
Student Educations Cumulative Gpa                                3.423567
Applications Count                                               1.042078
days after posted                                               23.448730
days before due                                                 45.370999
Employer Industry Name_Accounting                                0.007179
Employer Industry Name_Advertising, PR & Marketing               0.016684
Employer Industry Name_Aerospace                                 0.021281
Employer Industry Name_Automotive                                0.010170
Employer Industry Name_Biotech & Life Sciences                   0.038309
Employer Industry Name_CPG - Consumer Packaged Goods             0.002299
Employer Industry Name_Civil Engineering                         0.005380
Employer Industry Name_Commercial Banking & Credit               0.008368
Employer Industry Name_Computer Networ

In [31]:
student_grp_corr_post.mean()

days after posted    0.002157
days before due      0.009118
dtype: float64

In [30]:
student_grp_corr_post[student_grp_corr_post['days before due']>0.5]

,days after posted,days before due
days before due,0.039117,1.0
